In [1]:
%config InlineBackend.figure_format = 'retina'
%load_ext watermark
%watermark

Last updated: 2024-07-17T00:17:49.259828-04:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.17.2

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.5.0-44-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit



In [2]:
# Libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import warnings

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)

# Customize the font and title size
custom_params = {
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "font.family": "Arial"
}
sns.set_context("notebook", rc=custom_params)

# Set a color palette
sns.set_palette("Set2") 

# Set global option to display numbers in plain format
pd.set_option('display.float_format', '{:.2f}'.format)

warnings.filterwarnings('ignore')

In [3]:
## Auxiliary Functions

def detect_outliers(data, column):
    # Calculate Q1 and Q3
    Q1 = np.percentile(data[column], 25)
    Q3 = np.percentile(data[column], 75)

    # Calculate IQR
    IQR = Q3 - Q1

    # Define the outliers bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    data['Outlier'] = data[column].apply(lambda x: 'YES' if x < lower_bound or x > upper_bound else 'NO')

    return data

    
    # Use NumPy to create a boolean array indicating outliers
    #is_outlier = (data[column] < lower_bound) | (data[column] > upper_bound)
    # Convert the boolean array to 'YES'/'NO'
    #data['Outlier'] = np.where(is_outlier, 'YES', 'NO')
    #return data


def proportion_calc(data, column):
    tmp = data[column].value_counts().reset_index(name = 'count')
    tmp['prop'] = tmp['count']/tmp['count'].sum()*100
    return tmp

In [4]:
# Data 

CustomerData = pd.read_excel('DATA/CustomersData.xlsx')
Discount_Coupon = pd.read_csv('DATA/Discount_Coupon.csv')
Marketing_Spend = pd.read_csv('DATA/Marketing_Spend.csv')
Online_Sales = pd.read_csv('DATA/Online_Sales.csv')
Tax_amount = pd.read_excel('DATA/Tax_amount.xlsx')

In [5]:
display(CustomerData.head())
display(Discount_Coupon.head())
display(Marketing_Spend.head())
display(Online_Sales.head())
display(Tax_amount.head())

,CustomerID,Gender,Location,Tenure_Months
0,17850,M,Chicago,12
1,13047,M,California,43
2,12583,M,Chicago,33
3,13748,F,California,30
4,15100,M,California,49


,Month,Product_Category,Coupon_Code,Discount_pct
0,Jan,Apparel,SALE10,10
1,Feb,Apparel,SALE20,20
2,Mar,Apparel,SALE30,30
3,Jan,Nest-USA,ELEC10,10
4,Feb,Nest-USA,ELEC20,20


,Date,Offline_Spend,Online_Spend
0,1/1/2019,4500,2424.50
1,1/2/2019,4500,3480.36
2,1/3/2019,4500,1576.38
3,1/4/2019,4500,2928.55
4,1/5/2019,4500,4055.30


,CustomerID,Transaction_ID,Transaction_Date,Product_SKU,Product_Description,Product_Category,Quantity,Avg_Price,Delivery_Charges,Coupon_Status
0,17850,16679,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.50,Used
1,17850,16680,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.50,Used
2,17850,16681,1/1/2019,GGOEGFKQ020399,Google Laptop and Cell Phone Stickers,Office,1,2.05,6.50,Used
3,17850,16682,1/1/2019,GGOEGAAB010516,Google Men's 100% Cotton Short Sleeve Hero Tee...,Apparel,5,17.53,6.50,Not Used
4,17850,16682,1/1/2019,GGOEGBJL013999,Google Canvas Tote Natural/Navy,Bags,1,16.50,6.50,Used


,Product_Category,GST
0,Nest-USA,0.10
1,Office,0.10
2,Apparel,0.18
3,Bags,0.18
4,Drinkware,0.18


In [6]:
CustomerData.head()

,CustomerID,Gender,Location,Tenure_Months
0,17850,M,Chicago,12
1,13047,M,California,43
2,12583,M,Chicago,33
3,13748,F,California,30
4,15100,M,California,49


In [7]:
Discount_Coupon.head()

,Month,Product_Category,Coupon_Code,Discount_pct
0,Jan,Apparel,SALE10,10
1,Feb,Apparel,SALE20,20
2,Mar,Apparel,SALE30,30
3,Jan,Nest-USA,ELEC10,10
4,Feb,Nest-USA,ELEC20,20


In [8]:
Marketing_Spend.head()

,Date,Offline_Spend,Online_Spend
0,1/1/2019,4500,2424.50
1,1/2/2019,4500,3480.36
2,1/3/2019,4500,1576.38
3,1/4/2019,4500,2928.55
4,1/5/2019,4500,4055.30


In [9]:
Online_Sales.head()

,CustomerID,Transaction_ID,Transaction_Date,Product_SKU,Product_Description,Product_Category,Quantity,Avg_Price,Delivery_Charges,Coupon_Status
0,17850,16679,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.50,Used
1,17850,16680,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.50,Used
2,17850,16681,1/1/2019,GGOEGFKQ020399,Google Laptop and Cell Phone Stickers,Office,1,2.05,6.50,Used
3,17850,16682,1/1/2019,GGOEGAAB010516,Google Men's 100% Cotton Short Sleeve Hero Tee...,Apparel,5,17.53,6.50,Not Used
4,17850,16682,1/1/2019,GGOEGBJL013999,Google Canvas Tote Natural/Navy,Bags,1,16.50,6.50,Used


In [10]:
Tax_amount.head()

,Product_Category,GST
0,Nest-USA,0.10
1,Office,0.10
2,Apparel,0.18
3,Bags,0.18
4,Drinkware,0.18


# Business Questions

### 1. Calculate Invoice amount or sale_amount or revenue for each transaction and item level 
$$ Invoice Value =[\frac{Quantity*Avg\_price}{(1-Dicount\_pct)}*(1+GST)]+Delivery\_Charges$$

In [11]:
# Including the GST information

data_tmp = pd.merge(Online_Sales, Tax_amount, how='left', on='Product_Category')
data_tmp.head()

,CustomerID,Transaction_ID,Transaction_Date,Product_SKU,Product_Description,Product_Category,Quantity,Avg_Price,Delivery_Charges,Coupon_Status,GST
0,17850,16679,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.50,Used,0.10
1,17850,16680,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.50,Used,0.10
2,17850,16681,1/1/2019,GGOEGFKQ020399,Google Laptop and Cell Phone Stickers,Office,1,2.05,6.50,Used,0.10
3,17850,16682,1/1/2019,GGOEGAAB010516,Google Men's 100% Cotton Short Sleeve Hero Tee...,Apparel,5,17.53,6.50,Not Used,0.18
4,17850,16682,1/1/2019,GGOEGBJL013999,Google Canvas Tote Natural/Navy,Bags,1,16.50,6.50,Used,0.18


In [12]:
# Including discount_pct information
## ajusting the month name
data_tmp['Transaction_Date2'] = pd.to_datetime(data_tmp['Transaction_Date'])
data_tmp['Month'] = data_tmp['Transaction_Date2'].dt.strftime('%b')

## Merging the tables
data_tmp = pd.merge(data_tmp, Discount_Coupon,how='left', on='Month')

In [13]:
# Creating a new column to ajust the discount
# I will apply the discount if, just if, the coupon_status == Used.

data_tmp['Discount_pct_ajus'] = data_tmp.apply(lambda x: x['Discount_pct'] if x['Coupon_Status'] == 'Used' else 0, axis=1)

In [14]:
data_tmp

,CustomerID,Transaction_ID,Transaction_Date,Product_SKU,Product_Description,Product_Category_x,Quantity,Avg_Price,Delivery_Charges,Coupon_Status,GST,Transaction_Date2,Month,Product_Category_y,Coupon_Code,Discount_pct,Discount_pct_ajus
0,17850,16679,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.50,Used,0.10,2019-01-01,Jan,Apparel,SALE10,10,10
1,17850,16679,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.50,Used,0.10,2019-01-01,Jan,Nest-USA,ELEC10,10,10
2,17850,16679,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.50,Used,0.10,2019-01-01,Jan,Office,OFF10,10,10
3,17850,16679,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.50,Used,0.10,2019-01-01,Jan,Drinkware,EXTRA10,10,10
4,17850,16679,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.50,Used,0.10,2019-01-01,Jan,Lifestyle,EXTRA10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899703,14600,48497,12/31/2019,GGOENEBQ079099,Nest Protect Smoke + CO White Battery Alarm-USA,Nest-USA,4,80.52,19.99,Clicked,0.10,2019-12-31,Dec,Housewares,HOU30,30,0
899704,14600,48497,12/31/2019,GGOENEBQ079099,Nest Protect Smoke + CO White Battery Alarm-USA,Nest-USA,4,80.52,19.99,Clicked,0.10,2019-12-31,Dec,Accessories,ACC30,30,0
899705,14600,48497,12/31/2019,GGOENEBQ079099,Nest Protect Smoke + CO White Battery Alarm-USA,Nest-USA,4,80.52,19.99,Clicked,0.10,2019-12-31,Dec,Gift Cards,GC30,30,0
899706,14600,48497,12/31/2019,GGOENEBQ079099,Nest Protect Smoke + CO White Battery Alarm-USA,Nest-USA,4,80.52,19.99,Clicked,0.10,2019-12-31,Dec,Notebooks & Journals,NJ30,30,0


In [15]:
data_tmp['Invoice'] = (data_tmp['Quantity'] * data_tmp['Avg_Price'] / (1 - data_tmp['Discount_pct_ajus']/100)*(1+data_tmp['GST']) + data_tmp['Delivery_Charges'])

In [16]:
data_tmp['Invoice'].describe()

count   899708.00
mean       117.11
std        199.13
min          4.86
25%         22.18
50%         51.47
75%        169.62
max      13803.51
Name: Invoice, dtype: float64

In [17]:
print(f'The total invoice amount is ${data_tmp["Invoice"].sum():,.2f}')

The total invoice amount is $105,368,289.54


In [33]:
### The invoice transation for each product

invoice_product = pd.DataFrame(data_tmp.groupby('Product_Category_x')['Invoice'].agg('sum').reset_index(name = 'total'))
invoice_product['prop'] = invoice_product['total']/invoice_product['total'].sum()*100
invoice_product.sort_values('total', ascending=False)

### Graph Sankey

data_sankey = data_tmp.groupby(['Product_Category_x','Product_Description'])['Invoice'].agg('sum').reset_index()
data_sankey.columns = ['source', 'target', 'value']


### Tabela Resumo
qtde_product_cat = data_sankey['source'].value_counts().reset_index(name = 'qtde_items')
qtde_product_cat.columns = ['source','qtde_items']

invoice_product = pd.merge(invoice_product, qtde_product_cat, how='left', left_on='Product_Category_x', right_on='source')
invoice_product['avg_product_cateory'] = invoice_product['total']/invoice_product['qtde_items']
invoice_product.sort_values('total', ascending=False)


,Product_Category_x,total,prop,source,qtde_items,avg_product_cateory
16,Nest-USA,53654890.79,50.92,Nest-USA,9,5961654.53
2,Apparel,15914755.21,15.10,Apparel,211,75425.38
14,Nest,10423298.14,9.89,Nest,8,1302912.27
18,Office,7317974.85,6.95,Office,43,170185.46
6,Drinkware,5197631.20,4.93,Drinkware,28,185629.69
4,Bags,3707706.17,3.52,Bags,17,218100.36
12,Lifestyle,2353122.09,2.23,Lifestyle,16,147070.13
17,Notebooks & Journals,2306335.29,2.19,Notebooks & Journals,14,164738.23
15,Nest-Canada,1480478.47,1.41,Nest-Canada,5,296095.69
10,Headgear,1202263.82,1.14,Headgear,18,66792.43


In [34]:
import plotly.graph_objects as go

# Extract unique labels
all_labels = list(set(data_sankey['source'].tolist() + data_sankey['target'].tolist()))

# Create a mapping from label to index
label_to_index = {label: index for index, label in enumerate(all_labels)}

# Map the source and target labels to indices
data_sankey['source_id'] = data_sankey['source'].apply(lambda x: label_to_index[x])
data_sankey['target_id'] = data_sankey['target'].apply(lambda x: label_to_index[x])

# Create the link and node data for the Sankey diagram
link = dict(source=data_sankey['source_id'], target=data_sankey['target_id'], value=data_sankey['value'])
node = dict(label=all_labels, pad=15, thickness=20, line=dict(color="black", width=0.5))

# Create the Sankey diagram
sankey_data = go.Sankey(link=link, node=node)

# Create the figure
fig = go.Figure(data=sankey_data)

# Show the figure
fig.show()

## 2. Perform Detailed exploratory analysis  

### 2.1. Understanding how many customers acquired every month  